In [1]:
# https://www.jeffreythompson.org/collision-detection/table_of_contents.php

In [ ]:
# https://www.jeffreythompson.org/collision-detection/line-line.php

In [4]:

x1 = 0    # line controlled by mouse
y1 = 0
x2 = 10   # fixed end
y2 = 10

x3 = 100;  # static line
y3 = 300;
x4 = 500;
y4 = 100;




uA = ((x4-x3)*(y1-y3) - (y4-y3)*(x1-x3)) / ((y4-y3)*(x2-x1) - (x4-x3)*(y2-y1));

uB = ((x2-x1)*(y1-y3) - (y2-y1)*(x1-x3)) / ((y4-y3)*(x2-x1) - (x4-x3)*(y2-y1));





In [7]:
uA,uB

(23.333333333333332, 0.3333333333333333)

In [10]:
import numpy as np
import keras
from keras.layers import Lambda


p_pos = np.array([10,40])
p_dir = np.array([0.,1.])

2022-05-06 00:22:07.968569: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-06 00:22:07.968589: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [154]:
input_p = keras.layers.Input(shape=(8),batch_size=2)

#mid = keras.layers.Activation(keras.activations.backend.cos)(input_p)
#mid = input_p[0][0]-input_p[0][1]

outputs=[]

for pairidx,pair in enumerate(input_p):
    
    x1 = input_p[pairidx][0]   # line controlled by mouse
    y1 = input_p[pairidx][1]
    x2 = input_p[pairidx][2]   # fixed end
    y2 = input_p[pairidx][3]

    x3 = input_p[pairidx][4] # static line
    y3 = input_p[pairidx][5]
    x4 = input_p[pairidx][6]
    y4 = input_p[pairidx][7]



    uA = ((x4-x3)*(y1-y3) - (y4-y3)*(x1-x3)) / ((y4-y3)*(x2-x1) - (x4-x3)*(y2-y1));

    uB = ((x2-x1)*(y1-y3) - (y2-y1)*(x1-x3)) / ((y4-y3)*(x2-x1) - (x4-x3)*(y2-y1));
    outputs.append(keras.backend.stack([uA,uB]))
    


outs = keras.layers.concatenate(outputs)

m = keras.Model(inputs=input_p, outputs=outs)

In [151]:
mid.shape

TensorShape([])

ValueError: Exception encountered when calling layer "tf.__operators__.getitem_1807" (type SlicingOpLambda).

slice index 2 of dimension 0 out of bounds. for '{{node tf.__operators__.getitem_1807/strided_slice}} = StridedSlice[Index=DT_INT32, T=DT_FLOAT, begin_mask=0, ellipsis_mask=0, end_mask=0, new_axis_mask=0, shrink_axis_mask=1](Placeholder, tf.__operators__.getitem_1807/strided_slice/stack, tf.__operators__.getitem_1807/strided_slice/stack_1, tf.__operators__.getitem_1807/strided_slice/stack_2)' with input shapes: [2,2], [1], [1], [1] and with computed input tensors: input[1] = <2>, input[2] = <3>, input[3] = <1>.

Call arguments received:
  • tensor=tf.Tensor(shape=(2, 2), dtype=float32)
  • slice_spec=2
  • var=None

In [289]:
input_p = keras.layers.Input(shape=(2,2),batch_size=1)




n_pos = input_p[0][0]
n_dir = input_p[0][1]

n_dir = n_dir * 100
end_dir = n_pos + n_dir


first_half = keras.backend.stack([ [n_pos,end_dir] ])
#first_half = keras.layers.Concatenate([n_pos,end_dir,])
first_half = keras.layers.Reshape((1,4,))(first_half)

m = keras.Model(inputs=input_p, outputs= first_half)



In [290]:
n_pos

<KerasTensor: shape=(2,) dtype=float32 (created by layer 'tf.__operators__.getitem_1909')>

In [291]:
p_pos = np.array([10,40])
p_dir = np.array([0.,1.])


m.predict(np.array([[p_pos,p_dir]]))

array([[[ 10.,  40.,  10., 140.]]], dtype=float32)

In [ ]:
[0,50,100,50]

In [152]:
m.predict(np.array([[0,50,100,50, 70,100,70,0],[0,50,100,50, 70,100,70,0]]))

array([0.7, 0.5, 0.7, 0.5], dtype=float32)